In [12]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

# Parameters J3/K3
lookback_days = 63  # ~3 months
holding_days = 63   # ~3 months
data_dir = Path("stock_info")
price_data = {}

for file in data_dir.glob("*.csv"):
    try:
        df = pd.read_csv(file, parse_dates=["Date"])
        df = df.sort_values("Date").set_index("Date")
        
        # Remove dollar signs and convert to float
        df["Close/Last"] = df["Close/Last"].replace('[\$,]', '', regex=True).astype(float)

        price_data[file.stem] = df["Close/Last"]
        print(f"Loaded {file.name}, {len(df)} rows")

    except Exception as e:
        print(f"Failed to load {file.name}: {e}")

# Combine into one DataFrame
price_data_df = pd.DataFrame(price_data)
print(f"\nCombined DataFrame shape: {price_data_df.shape}")
print(price_data_df.head())

# Align to business days
prices = prices.asfreq("B")
prices = prices.dropna(axis=1, thresh=int(0.8 * len(prices)))

# Calculate returns
returns = prices.pct_change().dropna()

# Build portfolio strategy
future_returns = returns.shift(-holding_days)

portfolio_returns = []

# Use dates where we can apply lookback and holding period
rebalance_dates = returns.index[lookback_days : -holding_days : holding_days]

Loaded AAPL.csv, 1256 rows
Loaded GOOG.csv, 1256 rows
Loaded MSFT.csv, 1256 rows

Combined DataFrame shape: (1256, 3)
               AAPL     GOOG    MSFT
Date                                
2020-06-15  85.7475  70.9925  188.94
2020-06-16  88.0200  72.1360  193.57
2020-06-17  87.8975  72.5560  194.24
2020-06-18  87.9325  71.7980  196.32
2020-06-19  87.4300  71.5860  195.15


<>:20: SyntaxWarning: invalid escape sequence '\$'
<>:20: SyntaxWarning: invalid escape sequence '\$'
C:\Users\NickPC\AppData\Local\Temp\ipykernel_17824\330444096.py:20: SyntaxWarning: invalid escape sequence '\$'
  df["Close/Last"] = df["Close/Last"].replace('[\$,]', '', regex=True).astype(float)


In [13]:
print(f"Number of trading days: {len(prices.index)}")
print(f"Lookback days: {lookback_days}, Holding days: {holding_days}")
print(f"Valid iteration range: {len(prices.index[lookback_days : -holding_days])} days")

# Compute scores and returns for each rebalance date
for date in rebalance_dates:
    lookback_window = returns.loc[date - pd.Timedelta(days=lookback_days*2): date]
    if len(lookback_window) < lookback_days:
        continue

    momentum_scores = (prices.loc[date] / prices.loc[date - pd.Timedelta(days=lookback_days)]) - 1
    momentum_scores = momentum_scores.dropna()

    # Put into 10 ranks
    ranked = momentum_scores.rank(pct=True)
    deciles = pd.qcut(ranked, 10, labels=False)

    # Get returns over holding period
    end_date = date + pd.Timedelta(days=holding_days)
    if end_date not in future_returns.index:
        continue

    holding_return = future_returns.loc[date:end_date].mean()

    for decile in range(10):
        stocks = momentum_scores.index[deciles == decile]
        avg_return = holding_return[stocks].mean()
        portfolio_returns.append({
            "Date": date,
            "Decile": decile + 1,
            "Return": avg_return
        })

Number of trading days: 0
Lookback days: 63, Holding days: 63
Valid iteration range: 0 days


In [14]:
# Create Portfolio DataFrame
portfolio_df = pd.DataFrame(portfolio_returns)

print(portfolio_df.head())
print(portfolio_df.columns)

# Plot results
plt.figure(figsize=(10, 6))
sns.lineplot(
    data=portfolio_df.groupby("Decile")["Return"].mean().reset_index(),
    x="Decile", y="Return", marker="o"
)
plt.title("Average Holding Period Return by Momentum Decile (J3/K3)")
plt.ylabel("Average Return")
plt.xlabel("Momentum Decile (1 = Losers, 10 = Winners)")
plt.grid(True)
plt.tight_layout()
plt.show()

Empty DataFrame
Columns: []
Index: []
RangeIndex(start=0, stop=0, step=1)


KeyError: 'Decile'

<Figure size 1000x600 with 0 Axes>